In [ ]:
import os
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 🔐 Ton token Hugging Face (protège-le en pratique !)
# HF_TOKEN = ""
API_URL = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
# HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"}

# 🔁 Fonctions pour embeddings

def load_documents_from_folder(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                documents.append(file.read())
    return documents

def create_embeddings(documents, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(documents, convert_to_tensor=True)
    return embeddings

def search_relevant_document(question, documents, embeddings, model_name='all-MiniLM-L6-v2', threshold=0.4):
    model = SentenceTransformer(model_name)
    question_embedding = model.encode([question], convert_to_tensor=True)
    similarities = cosine_similarity(question_embedding.cpu().numpy(), embeddings.cpu().numpy())
    max_score = similarities.max()
    if max_score < threshold:
        return None
    best_doc_index = similarities.argmax()
    return documents[best_doc_index]

# 💡 Nouvelle version avec API Hugging Face

def answer_question_with_mistral_api(question, context):
    context = context.strip().replace("\n", " ")
    truncated_context = context[:1500]  # max ~1500-2000 chars pour éviter dépassement token

    prompt = (
        f"[INST] Réponds à la question suivante en te basant uniquement sur le contexte fourni.\n\n"
        f"Ne me demandes pas de consulter les séctions de la notice.\n\n"
        f"Contexte : {truncated_context}\n\n"
        f"Question : {question} [/INST]"
    )

    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 256,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": False,
            "return_full_text": False
        }
    }

    response = requests.post(API_URL, headers=HEADERS, json=payload)

    if response.status_code == 200:
        generated_text = response.json()[0]["generated_text"]
        return generated_text.strip()
    else:
        return f"Erreur API ({response.status_code}): {response.text}"

# 🧪 Exemple d'utilisation

folder_path = 'notices_txt'
question = " c'est quoi CIMETIDINE ARROW 200 mg ? "

documents = load_documents_from_folder(folder_path)
embeddings = create_embeddings(documents)
relevant_document = search_relevant_document(question, documents, embeddings)

if relevant_document:
    print("✅ Document pertinent trouvé.")
    print(relevant_document)
    answer = answer_question_with_mistral_api(question, relevant_document)
    print("\n🧠 Réponse :", answer)
else:
    print("❗ Aucun document pertinent trouvé pour cette question.")


/home/abdellatifunix/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-17 11:26:36.138130: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 11:26:36.460443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744881996.496830   13870 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744881996.504817   13870 cuda_blas.cc:1407] Un

✅ Document pertinent trouvé.
Dénomination du médicament
CIMETIDINE ARROW 200 mg, comprimé effervescent
Encadré
Veuillez lire attentivement cette notice avant de prendre ce médicament car elle contient des informations importantes pour vous.
·
Gardez cette notice. Vous pourriez avoir besoin de la relire.
·
Si vous avez dautres questions, interrogez votre médecin ou votre pharmacien.
·
Ce médicament vous a été personnellement prescrit. Ne le donnez pas à dautres personnes. Il pourrait leur être nocif, même si les signes de leur maladie sont identiques aux vôtres.
·
Si vous ressentez un quelconque effet indésirable, parlez-en à votre médecin ou votre pharmacien. Ceci sapplique aussi à tout effet indésirable qui ne serait pas mentionné dans cette notice.
Voir rubrique 4.
Que contient cette notice ?
1. Qu'est-ce que CIMETIDINE ARROW 200 mg, comprimé effervescent et dans quels cas est-il utilisé ?
2. Quelles sont les informations à connaître avant de prendre CIMETIDINE ARROW 200 mg, compr